# BILLE SUR GLISSIÈRE. EXPERIMENT

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import math

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import HTML

from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure, curdoc
from bokeh.models import Arrow, OpenHead, NormalHead, VeeHead
from bokeh.layouts import widgetbox
from bokeh.models.widgets import Slider
output_notebook()

### Import data from analysed video and video

In [ ]:
import io
import base64
from IPython.display import HTML

In [ ]:
video = io.open('data_bille/mirrored-points.mov', 'r+b').read()
encoded = base64.b64encode(video)
HTML(data='''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii')))

# iopub.data_rate

### Graphics

Trajectory and free parabola

In [ ]:
# Set path image for background
img_path = 'data_bille/background.png'

# Import data from the video
df = pd.read_csv('data_bille/data-05-04-19.csv', index_col=0)
df = df.rename({'VideoAnalysis: X (cm)': 'x', 'VideoAnalysis: Y (cm)':'y',
       'VideoAnalysis: X Velocity (cm/s)':'v_x', 'VideoAnalysis: Y Velocity (cm/s)':'v_y'}, axis='columns')

# Compute characteristics 
y_min = df['y'].min()
x_min = df['x'].min()
x_max = df['x'].max()
x_lowest = df.loc[df['y'] == y_min]['x'].iloc[0]

# Compute Radius of the loop
df_h = df.loc[df['x'] < 20]
df_h = df_h.loc[df_h['x'] > -20]
R = df_h['y'].max() - df_h['y'].min()
idx = df_h.loc[df_h['y'] == df_h['y'].min()]
R = R/2

In [ ]:
# Define parameters for calculations
g = 9.81
h = 2*R
omega = 20
to_rad = lambda x: np.pi*x/180.0
omegarad = to_rad(omega)

# Define equations
v_a = math.sqrt(2*(g*h - g*R*(1 + np.sin(omegarad)))) #Velocity at point A
n_a = (1/R) * v_a**2 - 9.8*np.sin(omegarad)


t = np.linspace(0, 2, 1000)
neg_t = np.linspace(0, -1, 100)

# Parametric eq. of parabola
x_p = lambda v, x, t: -v*np.sin(x)*t + R*np.cos(x)
y_p = lambda v, x, t: -0.5*g*t*t + v*np.cos(x)*t + R*(1 + np.sin(x))

# Parametric eq. of velocity
vx = lambda v, ang: np.linspace(x_p(v, ang, t)[0], x_p(v, ang, t)[0] - 0.5*v*np.sin(ang), 10) 
vy = lambda v, ang: np.linspace(y_p(v, ang, t)[0], y_p(v, ang, t)[0] + 0.5*v*np.cos(ang), 10) 


# Define data to plot
x = -df['x']
y = df['y'] - y_min
x0 = x.iloc[0]
y0 = y.iloc[0]

# Define track
slope2 = (y.iloc[6] - y.iloc[5]) / (x.iloc[6] - x.iloc[5])
get_x = lambda y_ini: x0 - (y0 - y_ini)*(1/slope2)

line_ini_y = np.linspace(2, y.iloc[0], 20)
line_ini_x = get_x(line_ini_y)

slope = np.arctan((y.iloc[6] - y.iloc[5]) / (x.iloc[6] - x.iloc[5]))
line_end_x = x.iloc[-1] - np.linspace(0, 40, 20)*np.cos(-slope-0.02)
line_end_y = y.iloc[-1] - np.linspace(0, 40, 20)*np.sin(-slope -0.02)

angle = np.linspace(0, 2*np.pi, 100)

In [ ]:
# Generate figure
p = figure(title="Trajectory", plot_height=425, plot_width=950, x_range=(-57,53), y_range=(-5,45),\
           background_fill_color='#ffffff')
# Set background
p.image_url(url=[img_path], x=-56.5, y=45, w=110, h=45, alpha=0.2, angle=-np.pi*1/180)


# Static part of the plot:
# Plot track
p.line(line_ini_x, line_ini_y, color='#635c74', alpha=0.7, line_width=1.5, legend='Track')
#p.line(line_end_x, line_end_y, color='#635c74', alpha=0.7, line_width=1.5, legend='Track')
p.line(R*np.sin(angle), R + R*np.cos(angle), color='#635c74', alpha=0.6, line_width=1, legend='Track')
# Plot center of the loop
p.circle(0,R,size=3, fill_color='#635c74', line_color='#635c74', legend='Center')


# Variables:
# Initial point
p_ini = p.circle([get_x(h)], [h], size=6, fill_color='#e32020', line_color='#e32020', legend='Initial Point')

# Point A
p_a = p.circle([x_p(v_a, omegarad, t)[0]],[y_p(v_a, omegarad, t)[0]], size=6, \
               legend='A', line_color='#e32020', fill_color='#e32020')

#Parabola
parab = p.line(x_p(v_a, omegarad, t), y_p(v_a, omegarad, t), color='#e32020', line_width=1.5, alpha=0.8, \
               legend='Parabola')

parab_neg = p.line(x_p(v_a, omegarad, neg_t), y_p(v_a, omegarad, neg_t), color='#e32020', line_width=1.5, alpha=0.8, \
               legend='Parabola')

#Lines for angle omega
l_h = p.line(np.linspace(0,R, 100), [R]*100, line_width=1, color='#000000', alpha=0.2)
l_v = p.line(np.linspace(0, x_p(v_a, omegarad, t)[0], 100), np.linspace(R,y_p(v_a, omegarad, t)[0],100) , \
            line_width=1 ,color='#000000', alpha=0.2)

f_g = p.line([x_p(v_a, omegarad, t)[0]]*10, np.linspace(y_p(v_a, omegarad, t)[0], 0.8*y_p(v_a, omegarad, t)[0], 10), \
             line_width=1, color='#178717', alpha=0.8, legend='mg')
a_u_f = p.line(np.linspace(x_p(v_a, omegarad, t)[0], 3+x_p(v_a, omegarad, t)[0], 10), \
              np.linspace(0.8*y_p(v_a, omegarad, t)[0], 3 +y_p(v_a, omegarad, t)[0], 10),\
              line_width=1, color='#178717', alpha=0.8, legend='mg')
a_d_f = p.line(np.linspace(x_p(v_a, omegarad, t)[0], -3 + x_p(v_a, omegarad, t)[0], 10), \
              np.linspace(0.8*y_p(v_a, omegarad, t)[0], 3 +y_p(v_a, omegarad, t)[0], 10),\
              line_width=1, color='#178717', alpha=0.8, legend='mg')


f_n = p.line(np.linspace(x_p(v_a, omegarad, t)[0], x_p(v_a, omegarad, t)[0] - n_a*np.cos(omegarad), 10), \
             np.linspace(y_p(v_a, omegarad, t)[0], y_p(v_a, omegarad, t)[0] - n_a*np.sin(omegarad), 10), \
             line_width=1.5, color='#064d06', alpha=0.8, legend='N')
a_u_n = p.line(np.linspace(x_p(v_a, omegarad, t)[0] - n_a*np.cos(omegarad), x_p(v_a, omegarad, t)[0] - n_a*np.cos(omegarad) + 0.1*n_a*np.cos(omegarad - np.pi*20/180), 10), \
             np.linspace(y_p(v_a, omegarad, t)[0] - n_a*np.sin(omegarad) , y_p(v_a, omegarad, t)[0] - n_a*np.sin(omegarad)  + 0.1*n_a*np.sin(omegarad - np.pi*20/180), 10), \
             line_width=1.5, color='#064d06', alpha=0.8, legend='N')

a_d_n = p.line(np.linspace(x_p(v_a, omegarad, t)[0] - n_a*np.cos(omegarad), x_p(v_a, omegarad, t)[0] - n_a*np.cos(omegarad) + 0.1*n_a*np.cos(omegarad + np.pi*20/180), 10), \
             np.linspace(y_p(v_a, omegarad, t)[0] - n_a*np.sin(omegarad) , y_p(v_a, omegarad, t)[0] - n_a*np.sin(omegarad)  + 0.1*n_a*np.sin(omegarad + np.pi*20/180), 10), \
             line_width=1.5, color='#064d06', alpha=0.8, legend='N')




# Velocity vector
v_x, v_y = vx(v_a, omegarad), vy(v_a, omegarad)
v = p.line(v_x, v_y, color='#01B9FF', legend='Velocity') 
a_u = p.line(np.linspace(v_x[-1], v_x[-1] + 0.1*v_a*np.cos(omegarad - np.pi*70/180), 10),\
            np.linspace(v_y[-1], v_y[-1] + 0.1*v_a*np.sin(omegarad - np.pi*70/180), 10), line_width=2)

a_d = p.line(np.linspace(v_x[-1], v_x[-1] - 0.1*v_a*np.cos(omegarad + np.pi*70/180), 10),\
            np.linspace(v_y[-1], v_y[-1] - 0.1*v_a*np.sin(omegarad + np.pi*70/180), 10), line_width=2)

# Trajectory corresponding to the video
traj = p.line(x, y-0.25, color="#3743ec", line_width=1, line_dash='dashed', alpha=0.4, legend='Trajectory of the video')


def update(Initial_height, omega, info):
    h = Initial_height*R
    omegarad = to_rad(omega)
    
    v_a = math.sqrt(2*(g*h - g*R*(1 + np.sin(omegarad))))
    n_a = np.maximum((1/R) * v_a**2 - 9.8*np.sin(omegarad), 0) #if negative it will not plot the normal force
    
    v_x, v_y = vx(v_a, omegarad), vy(v_a, omegarad)

    x_parab = x_p(v_a, omegarad, t)
    y_parab = y_p(v_a, omegarad, t)

    parab.data_source.data['x'] = x_parab
    parab.data_source.data['y'] = y_parab
    
    parab_neg.data_source.data['x'] = x_p(v_a, omegarad, neg_t)
    parab_neg.data_source.data['y'] = y_p(v_a, omegarad, neg_t)

    p_ini.data_source.data['x'] = [x0 - (y0 - h)*(1/slope2)]
    p_ini.data_source.data['y'] = [h]

    p_a.data_source.data['x'] = [x_parab[0]]
    p_a.data_source.data['y'] = [y_parab[0]]
    
    f_g.data_source.data['x'] = [x_parab[0]]*10
    f_g.data_source.data['y'] = np.linspace(y_parab[0], -9.8 + y_parab[0], 10)
    a_u_f.data_source.data['x'] = np.linspace(x_parab[0], 0.5 + x_parab[0], 10)
    a_u_f.data_source.data['y'] = np.linspace(-9.8 + y_parab[0], 0.6 -9.8 + y_parab[0], 10)
    a_d_f.data_source.data['x'] = np.linspace(x_parab[0], -0.5 + x_parab[0], 10)
    a_d_f.data_source.data['y'] = np.linspace(-9.8 + y_parab[0], 0.6 -9.8 + y_parab[0], 10)
    
    f_n.data_source.data['x'] = np.linspace(x_parab[0], x_parab[0] - n_a*np.cos(omegarad), 10)
    f_n.data_source.data['y'] = np.linspace(y_parab[0], y_parab[0] - n_a*np.sin(omegarad), 10)
    a_u_n.data_source.data['x'] = np.linspace(x_parab[0] - n_a*np.cos(omegarad), x_parab[0] - n_a*np.cos(omegarad) + 0.1*n_a*np.cos(omegarad - np.pi*20/180), 10)
    a_u_n.data_source.data['y'] = np.linspace(y_parab[0] - n_a*np.sin(omegarad), y_parab[0] - n_a*np.sin(omegarad) + 0.1*n_a*np.sin(omegarad - np.pi*20/180), 10)
    a_d_n.data_source.data['x'] = np.linspace(x_parab[0] - n_a*np.cos(omegarad), x_parab[0] - n_a*np.cos(omegarad) + 0.1*n_a*np.cos(omegarad + np.pi*20/180), 10)
    a_d_n.data_source.data['y'] = np.linspace(y_parab[0] - n_a*np.sin(omegarad), y_parab[0] - n_a*np.sin(omegarad) + 0.1*n_a*np.sin(omegarad + np.pi*20/180), 10)
    
    
    l_v.data_source.data['x'] = np.linspace(0, x_parab[0], 100)
    l_v.data_source.data['y'] = np.linspace(R, y_parab[0],100)
    
    v.data_source.data['x'] = np.linspace(x_parab[0], x_parab[0] - 0.5*v_a*np.sin(omegarad), 10)
    v.data_source.data['y'] = np.linspace(y_parab[0], y_parab[0] + 0.5*v_a*np.cos(omegarad), 10)
    a_u.data_source.data['x'] = np.linspace(v.data_source.data['x'][-1],v.data_source.data['x'][-1] + 0.1*v_a*np.cos(omegarad - np.pi*70/180), 10)
    a_u.data_source.data['y'] = np.linspace(v.data_source.data['y'][-1], v.data_source.data['y'][-1] + 0.1*v_a*np.sin(omegarad - np.pi*70/180), 10)
    
    a_d.data_source.data['x'] = np.linspace(v.data_source.data['x'][-1],v.data_source.data['x'][-1] - 0.1*v_a*np.cos(omegarad + np.pi*70/180), 10)
    a_d.data_source.data['y'] = np.linspace(v.data_source.data['y'][-1], v.data_source.data['y'][-1] - 0.1*v_a*np.sin(omegarad + np.pi*70/180), 10)

    p.legend.click_policy="hide"
    if info:
        print('Velocity  : {:3.3} cm/s'.format(v_a))
        print('Velocity x: {:3.3} cm/s'.format(v_a*np.cos(omegarad)))
        print('Velocity y: {:3.3} cm/s'.format(v_a*np.sin(omegarad)))
        
        print('Point A.       Coordinates: x: {:3.3}, y: {:3.3}'.format(round(x_parab[0], 3), round(y_parab[0],3)))
        print('Initial point. Coordinates: x: {:3.3}, y: {:3.3}'.format(x0 - (y0 - h)*(1/slope2), h))
        
        if n_a > 0:
            print('Point not yet disconnected')
        else:
            print('Point already disconnected')

        
    push_notebook()

doc = curdoc()
doc.add_root(p)
show(p, notebook_handle=True)

slider_h = widgets.FloatSlider(min=2, max=3, step=0.05, value="2", labels=["Initial Height"])
info = widgets.Checkbox(value=False,description="Show information details",disabled=False)


interact(update, Initial_height=slider_h, omega=(0, 90), info=info);
print('Warning: the value of the initial height is expressed in R units')